In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.3"

In [ ]:
using System;
using System.IO;
using System.Net;
using System.Net.Sockets;
using System.Text;
using System.Threading.Tasks;
using System.Net.Http;
using Newtonsoft.Json.Linq;

class McpServer
{
    private TcpListener listener;
    private const int Port = 4000;
    private const string BingApiKey = "YOUR_BING_API_KEY";

    public McpServer()
    {
        listener = new TcpListener(IPAddress.Any, Port);
    }

    public async Task StartAsync()
    {
        listener.Start();
        Console.WriteLine($"MCP Server started on port {Port}");

        while (true)
        {
            var client = await listener.AcceptTcpClientAsync();
            _ = HandleClientAsync(client);
        }
    }

    private async Task HandleClientAsync(TcpClient client)
    {
        using (client)
        {
            var stream = client.GetStream();
            var buffer = new byte[4096];
            int bytesRead = await stream.ReadAsync(buffer, 0, buffer.Length);
            string request = Encoding.UTF8.GetString(buffer, 0, bytesRead);

            // Simple MCP message parsing: expecting "mcp-websearch <query>"
            if (request.StartsWith("mcp-websearch "))
            {
                string query = request.Substring("mcp-websearch ".Length).Trim();
                string result = await WebSearchAsync(query);

                string response = $"mcp-websearch-result {result}";
                byte[] responseBytes = Encoding.UTF8.GetBytes(response);
                await stream.WriteAsync(responseBytes, 0, responseBytes.Length);
            }
            else
            {
                string response = "mcp-error Unknown command";
                byte[] responseBytes = Encoding.UTF8.GetBytes(response);
                await stream.WriteAsync(responseBytes, 0, responseBytes.Length);
            }
        }
    }

    private async Task<string> WebSearchAsync(string query)
    {
        using (var httpClient = new HttpClient())
        {
            string url = $"https://api.bing.microsoft.com/v7.0/search?q={Uri.EscapeDataString(query)}";
            httpClient.DefaultRequestHeaders.Add("Ocp-Apim-Subscription-Key", BingApiKey);

            var response = await httpClient.GetAsync(url);
            string json = await response.Content.ReadAsStringAsync();

            var obj = JObject.Parse(json);
            var firstResult = obj["webPages"]?["value"]?[0]?["snippet"]?.ToString() ?? "No results found.";
            return firstResult;
        }
    }

    public static async Task Main(string[] args)
    {
        var server = new McpServer();
        await server.StartAsync();
    }
}

await McpServer.Main([]);